## Часть 1
Напишите скрипт, который скачивает все данные прошедших губернаторских выборов для всех избирательных участков. 

Входная точка [по ссылке](http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222). Затем нужно перейти на сайты территориальных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название ТИК (территориальная избирательная комиссия, некоторое объединение избирательных участков на определенной территории)
- номер УИК (избирательный участок)
- 14 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

In [1]:
import pandas as pd
import lxml.html
import lxml.etree
import requests
import re

In [2]:
url= requests.get("http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222")
page = lxml.html.fromstring(url.text)
df = pd.DataFrame()

In [3]:
com = page.xpath('//a[@style = "text-decoration: none"]')

for i in range(30):
    url= requests.get(com[i].attrib['href'])
    page = lxml.html.fromstring(url.text)
    table = []

    uik = page.xpath('/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[1]/td//text()')
    table.append([com[i].text]*len(uik))
    table.append(uik)
    
    col_names = ['Комиссия', 'УИК']
    
    for tik in range(2,13):
        row = page.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[{tik}]/td//b/text()')
        table.append(row)
        
        col_name = page.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[1]/table/tr[{tik}]/td[@align = "left"]/nobr/text()')
        col_names.append(col_name[0])
        
    for cand in range(14,17):
        row = page.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[{cand}]/td//b/text()')
        table.append(row)
        
        row = page.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[{cand}]/td')
    
        format_p = re.compile(r'(.+)%')
        row = list(map(lambda x:format_p.findall(x.text_content())[0],row))
        table.append(row)
        
        col_name = page.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[1]/table/tr[{cand}]/td[@align = "left"]/nobr/text()')
        col_names.append(col_name[0])
        col_names.append(col_name[0] + ' %')

    df = df.append(pd.DataFrame(table).transpose(),ignore_index = True)
    
df.columns = col_names
#df.to_csv('elections.csv')
df.head()   

,Комиссия,УИК,"Число избирателей, внесенных в список избирателей на момент окончания голосования","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования",Число погашенных избирательных бюллетеней,"Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования","Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования",Число недействительных избирательных бюллетеней,Число действительных избирательных бюллетеней,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Амосов Михаил Иванович,Амосов Михаил Иванович %,Беглов Александр Дмитриевич,Беглов Александр Дмитриевич %,Тихонова Надежда Геннадьевна,Тихонова Надежда Геннадьевна %
0,Территориальная избирательная комиссия №1,УИК №1,1803,1200,587,11,602,11,587,23,575,0,0,110,18.39,345,57.69,120,20.07
1,Территориальная избирательная комиссия №1,УИК №2,1466,1100,433,14,653,14,430,19,425,0,0,53,11.94,326,73.42,46,10.36
2,Территориальная избирательная комиссия №1,УИК №3,2092,1600,576,22,1002,22,576,15,583,0,0,155,25.92,332,55.52,96,16.05
3,Территориальная избирательная комиссия №1,УИК №4,1056,1000,318,4,678,4,318,13,309,0,0,67,20.81,171,53.11,71,22.05
4,Территориальная избирательная комиссия №1,УИК №5,1827,1400,495,8,897,8,493,16,485,0,0,137,27.35,266,53.09,82,16.37
